In [36]:
import pandas as pd
import numpy as np
import pickle

In [37]:
df_epovvc = pd.read_pickle(r"Q:\Meine Bibliotheken\Research\Projects\Ausgründungen\02_Data\02_Webdata2IPC\df_epovvc_s1.pkl")

In [38]:
df_epovvc.head(3)

,person_id,crefo,equal,bestcrefo,appln_id,appln_kind,appln_fili,inpadoc_fa,earliest_f,granted,technology,ipc,text,links
0,20,2150355978,1,1,16996827,A,1993-04-07,11221689,1992-04-07,Y,2,H04J 3/00,[->p<-] Intellectual Property & Standards [->p...,"[philips.com,twitter.com,youtube.com,plus.goog..."
1,20,2150355978,1,1,16996827,A,1993-04-07,11221689,1992-04-07,Y,2,H04J 3/06,[->p<-] Intellectual Property & Standards [->p...,"[philips.com,twitter.com,youtube.com,plus.goog..."
2,20,2150355978,1,1,16996827,A,1993-04-07,11221689,1992-04-07,Y,2,H04L 5/22,[->p<-] Intellectual Property & Standards [->p...,"[philips.com,twitter.com,youtube.com,plus.goog..."


In [42]:
# Function to extract relevant data
def func_extractor(df, cols, Y, X):
    """Prepare data for model training. Choosing features X and target variable Y. Select variables accordingly and 
    conduct further preprocessing steps."""
    
    # Select variables and drop duplicated rows
    res = df.loc[:, cols].drop_duplicates()
    dim1 = len(res.crefo.unique())
    
    # Drop rows with missing values in features
    res = res.loc[res[X].notnull(), :]
    dim2 = len(res.crefo.unique())
    print(str(dim1 - dim2) + ' samples dropped due to missings in X.')
    
    # Drop rows with missing values in target
    res = res.loc[res[Y].notnull(), :]
    dim3 = len(res.crefo.unique())
    print(str(dim2 - dim3) + ' further samples dropped due to missings in Y.')
    print(str(dim3) + ' samples remain.')
    
    # Calculate number of distinct classes
    n_classes = len(res[Y].unique())
    print('There are ' + str(n_classes) + ' distinct classes.')
    
    # Prepare target as multiple-hot vector of length = n_classes
    res_agg = res.groupby('crefo')
    y = res_agg[Y].apply(lambda x: np.eye(n_classes)[np.array([i-1 for i in x], dtype=int)].sum(axis=0, dtype=int))
    
    x = res_agg.agg(dict(text = lambda x: x.head(1)))
    
    df_train = pd.concat([y, x], axis=1)
    
    return df_train

In [40]:
df_train = func_extractor(df_epovvc, cols = ['crefo', 'technology', 'text'], Y = 'technology', X = 'text')

22296 samples dropped due to missings in X.
5 further samples dropped due to missings in Y.
14567 samples remain.
There are 35 distinct classes.


TypeError: list() takes no keyword arguments

In [35]:
# Write data
with open(r'Q:\Meine Bibliotheken\Research\Projects\Ausgründungen\02_Data\02_Webdata2IPC\df_train.pkl', 'wb') as f:
    pickle.dump(obj=df_train, file=f)